1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Набор данных: Heart Disease Data Set (https://archive.ics.uci.edu/ml/datasets/Heart+Disease)

Анализ анализ наличия сердечно-сосудитстых заболеваний.

In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
df = pd.read_csv('heart.csv')
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [20]:
df.loc[df['age'] <= 20, 'age'] = 1
df.loc[(df['age'] > 20) & (df['age'] <= 35), 'age'] = 2
df.loc[(df['age'] > 35) & (df['age'] <= 50), 'age'] = 3
df.loc[(df['age'] > 50) & (df['age'] <= 65), 'age'] = 4 
df.loc[(df['age'] > 65) & (df['age'] <= 80), 'age'] = 5 
df.loc[df['age'] >= 80, 'age'] = 6
df['age'].value_counts()

4    175
3     88
5     33
2      7
Name: age, dtype: int64

До 20 лет - никого. Группа риска - от 50 до 65 лет. А после 80 лет нет данных.

In [27]:
# Аналитика по давлению

cat_1 = len(df.loc[df['trestbps'] < 100])  #гипотония
cat_2 = len(df.loc[(df['trestbps'] >= 100) & (df['trestbps'] < 130)]) #норма
cat_3 = len(df.loc[(df['trestbps'] >= 130) & (df['trestbps'] < 140)]) #повышенное
cat_4 = len(df.loc[(df['trestbps'] >= 140) & (df['trestbps'] < 160)])  #гипертония 1 степени
cat_5 = len(df.loc[(df['trestbps'] >= 160) & (df['trestbps'] < 180)])  #гипертония 2 степени
cat_6 = len(df.loc[df['trestbps'] >= 180])  #гипертония 3 степени

df.loc[df['trestbps'] < 100, 'trestbps'] = 1 
df.loc[(df['trestbps'] >= 100) & (df['trestbps'] < 130), 'trestbps'] = 2
df.loc[(df['trestbps'] >= 130) & (df['trestbps'] < 140), 'trestbps'] = 3
df.loc[(df['trestbps'] >= 140) & (df['trestbps'] < 160), 'trestbps'] = 4
df.loc[(df['trestbps'] >= 160) & (df['trestbps'] < 180), 'trestbps'] = 5
df.loc[df['trestbps'] >= 180, 'trestbps'] = 6
df['trestbps'].value_counts()

2    133
4     71
3     71
5     21
6      5
1      2
Name: trestbps, dtype: int64

In [30]:
# Холестирин

df.loc[df['chol'] < 200, 'chol'] = 1 # желательный 
df.loc[(df['chol'] >= 200) & (df['chol'] < 240), 'chol'] = 2 #  пограничный
df.loc[df['chol'] >= 240, 'chol'] = 3 #  высокий
df['chol'].value_counts()

3    155
2     98
1     50
Name: chol, dtype: int64

Разбиваем выборку на тренировочную и тестовую части и обучаем модель.

In [32]:
from sklearn.model_selection import train_test_split

x_data = df.iloc[:,:-1]
y_data = df.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [33]:
model = GradientBoostingClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [35]:
# Проверка качества

results = {
    'approach': [],
    'f1': [],
    'roc': [],
    'recall':[],
    'precision':[]
}

from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 



results['approach'].append('gbc')
results['f1'].append(f1_score(y_test, y_predict))
results['roc'].append(roc_auc_score(y_test, y_predict))
results['recall'].append(recall_score(y_test, y_predict, average='binary'))
results['precision'].append(precision_score(y_test, y_predict, average='binary'))

evaluate_results(y_test, y_predict)

Classification results:
f1: 75.76%
roc: 73.66%
recall: 80.65%
precision: 71.43%


In [37]:
# PU learning

mod_data = df.copy()

pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 42/165 as positives and unlabeling the rest


In [38]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    261
 1     42
Name: class_test, dtype: int64


In [39]:
mod_data.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,class_test
0,4,1,3,4,2,1,0,150,0,2.3,0,0,1,1,-1
1,3,1,2,3,3,0,1,187,0,3.5,0,0,2,1,-1
2,3,0,1,3,2,0,0,172,0,1.4,2,0,2,1,-1
3,4,1,1,2,2,0,1,178,0,0.8,2,0,2,1,1
4,4,0,0,2,3,0,1,163,1,0.6,2,0,2,1,-1
5,4,1,0,4,1,0,1,148,0,0.4,1,0,1,1,-1
6,4,0,1,4,3,0,0,153,0,1.3,1,0,2,1,1
7,3,1,1,2,3,0,1,173,0,0.0,2,0,3,1,-1
8,4,1,2,5,1,1,1,162,0,0.5,2,0,3,1,-1
9,4,1,2,4,1,0,1,174,0,1.6,2,0,2,1,-1


In [40]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [41]:
# random negative sampling 25%

mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(42, 15) (42, 15)


In [42]:
model = GradientBoostingClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('gbc_25%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 74.59%
roc: 72.81%
recall: 89.22%
precision: 64.08%


In [44]:
# random negative sampling 10%

mod_data = df.copy()

pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.1 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 17/165 as positives and unlabeling the rest


In [45]:
# Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    286
 1     17
Name: class_test, dtype: int64


In [46]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(17, 15) (17, 15)


In [47]:
model = GradientBoostingClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('gbc_10%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 77.50%
roc: 72.40%
recall: 87.32%
precision: 69.66%


In [49]:
# random negative sampling 50%

mod_data = df.copy()

pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.5 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 83/165 as positives and unlabeling the rest


In [50]:
# Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    220
 1     83
Name: class_test, dtype: int64


In [51]:
mod_data.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,class_test
0,4,1,3,4,2,1,0,150,0,2.3,0,0,1,1,1
1,3,1,2,3,3,0,1,187,0,3.5,0,0,2,1,-1
2,3,0,1,3,2,0,0,172,0,1.4,2,0,2,1,1
3,4,1,1,2,2,0,1,178,0,0.8,2,0,2,1,1
4,4,0,0,2,3,0,1,163,1,0.6,2,0,2,1,-1
5,4,1,0,4,1,0,1,148,0,0.4,1,0,1,1,-1
6,4,0,1,4,3,0,0,153,0,1.3,1,0,2,1,1
7,3,1,1,2,3,0,1,173,0,0.0,2,0,3,1,1
8,4,1,2,5,1,1,1,162,0,0.5,2,0,3,1,-1
9,4,1,2,4,1,0,1,174,0,1.6,2,0,2,1,1


In [52]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(83, 15) (83, 15)


In [53]:
model = GradientBoostingClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('gbc_50%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 71.67%
roc: 81.12%
recall: 97.73%
precision: 56.58%


In [54]:
pd.DataFrame(data=results)

,approach,f1,roc,recall,precision
0,gbc,0.757576,0.736559,0.806452,0.714286
1,gbc_25%_rns,0.745902,0.728130,0.892157,0.640845
2,gbc_10%_rns,0.775000,0.724021,0.873239,0.696629
3,gbc_50%_rns,0.716667,0.811217,0.977273,0.565789
